We initialize Python imports and opens a DuckDB connection that every later cell reuses

In [ ]:
import duckdb
import pandas as pd

from pathlib import Path

cwd = Path.cwd()

root = None
for p in [cwd] + list(cwd.parents):
    if (p / "db").exists():
        root = p
        break

if root is None:
    raise FileNotFoundError("Could not find a 'db' folder above the current working directory")

DB_PATH = root / "db" / "nflpa.duckdb"

print("Using DB_PATH", DB_PATH)

con = duckdb.connect(str(DB_PATH))

con.execute("PRAGMA threads=4")
con.execute("PRAGMA memory_limit='4GB'")

Quick sanity check to confirm that the target table exists in the database and contains the necessary special teams metrics from the previous step to ensure the environment is ready for statistical analysis

In [ ]:
required_tables = ["team_week_panel", "schedules"]
existing = set(con.execute("SHOW TABLES").df()["name"].tolist())
missing_tables = [t for t in required_tables if t not in existing]

print("Missing tables", missing_tables)
print("OK" if not missing_tables else "STOP, fix missing tables before continuing")

panel_cols_df = con.execute("DESCRIBE team_week_panel").df()
panel_cols = set(panel_cols_df["column_name"].tolist())

required_cols = [
    "season",
    "week",
    "game_id",
    "ST_Load_All_w",
    "ST_Load_ScoreLinked_w",
    "ST_Load_NonScore_w",
]

missing_cols = [c for c in required_cols if c not in panel_cols]

print("Missing required columns in team_week_panel", missing_cols)
print("OK" if not missing_cols else "STOP, Step 4 outputs missing from panel")

We detect the primary team join key by inspecting the panel schema, allowing the downstream processing logic to remain agnostic to naming conventions between different data versions

In [ ]:
if "team_id" in panel_cols:
    PANEL_TEAM_COL = "team_id"
elif "team" in panel_cols:
    PANEL_TEAM_COL = "team"
else:
    raise ValueError("Could not find a team column in team_week_panel")

print("Using panel team column", PANEL_TEAM_COL)

We create a base view that identifies regular season game weeks to ensure that season-level team statistics are not skewed by bye weeks or post-season data

In [ ]:
con.execute(f"""
CREATE OR REPLACE TEMP VIEW panel_step5_base AS
SELECT
  p.*,
  s.game_type AS sched_game_type,
  CASE WHEN p.game_id IS NOT NULL AND s.game_type = 'REG' THEN 1 ELSE 0 END AS is_reg_game_week
FROM team_week_panel p
LEFT JOIN schedules s
  ON p.game_id = s.game_id
""")

Quick sanity check to confirm that the schedule merge didn't accidentally multiply our data rows and also that every game week is correctly labeled as a regular or post-season game

In [ ]:
con.execute(f"""
SELECT
  (SELECT COUNT(*) FROM team_week_panel) AS panel_rows,
  (SELECT COUNT(*) FROM panel_step5_base) AS base_rows,
  (SELECT COUNT(DISTINCT season || '-' || week || '-' || {PANEL_TEAM_COL}) FROM team_week_panel) AS panel_distinct_team_weeks,
  (SELECT COUNT(DISTINCT season || '-' || week || '-' || {PANEL_TEAM_COL}) FROM panel_step5_base) AS base_distinct_team_weeks,
  SUM(CASE WHEN game_id IS NOT NULL AND sched_game_type IS NULL THEN 1 ELSE 0 END) AS games_missing_schedule_type
FROM panel_step5_base
""").df()

We compute the mean and standard deviation for every workload category at the team-season level, using only active regular season weeks, and persist the results in a reference table for future normalization

In [ ]:
con.execute(f"""
CREATE OR REPLACE TABLE team_season_st_stats AS
SELECT
  season,
  {PANEL_TEAM_COL} AS team,

  AVG(ST_Load_All_w) AS mean_ST_All,
  STDDEV_SAMP(ST_Load_All_w) AS sd_ST_All,

  AVG(ST_Load_ScoreLinked_w) AS mean_ST_ScoreLinked,
  STDDEV_SAMP(ST_Load_ScoreLinked_w) AS sd_ST_ScoreLinked,

  AVG(ST_Load_NonScore_w) AS mean_ST_NonScore,
  STDDEV_SAMP(ST_Load_NonScore_w) AS sd_ST_NonScore,

  COUNT(*) AS n_game_weeks
FROM panel_step5_base
WHERE is_reg_game_week = 1
GROUP BY season, {PANEL_TEAM_COL}
""")

Quick sanity check to confirm that each team-season has a unique entry in the summary table, ensuring the game counts align with a typical season length and that the standard deviation is successfully calculated

In [ ]:
con.execute("""
SELECT
  COUNT(*) AS rows,
  COUNT(DISTINCT season || '-' || team) AS distinct_team_seasons,
  MIN(n_game_weeks) AS min_game_weeks,
  MAX(n_game_weeks) AS max_game_weeks,
  SUM(CASE WHEN sd_ST_All IS NULL OR sd_ST_All = 0 THEN 1 ELSE 0 END) AS zero_or_null_sd_all,
  SUM(CASE WHEN sd_ST_NonScore IS NULL OR sd_ST_NonScore = 0 THEN 1 ELSE 0 END) AS zero_or_null_sd_nonscore
FROM team_season_st_stats
""").df()

Quick sanity check to confirm that we can catch cases where the standard deviation is zero or missing, ensuring we don't cause errors when dividing to calculate z-scores

In [ ]:
con.execute("""
SELECT
  season,
  team,
  n_game_weeks,
  mean_ST_All,
  sd_ST_All,
  mean_ST_NonScore,
  sd_ST_NonScore
FROM team_season_st_stats
WHERE sd_ST_All IS NULL OR sd_ST_All = 0 OR sd_ST_NonScore IS NULL OR sd_ST_NonScore = 0
ORDER BY season, team
""").df()

We join the team season statistics back to the individual game-week rows and calculate the z-scores, defaulting to zero if the standard deviation is zero or if the week isn't a regular season game to keep the data clean

In [ ]:
con.execute(f"""
CREATE OR REPLACE TEMP VIEW panel_step5_z AS
SELECT
  p.*,

  ts.mean_ST_All,
  ts.sd_ST_All,
  ts.mean_ST_ScoreLinked,
  ts.sd_ST_ScoreLinked,
  ts.mean_ST_NonScore,
  ts.sd_ST_NonScore,
  ts.n_game_weeks,

  CASE
    WHEN ts.mean_ST_All IS NOT NULL AND ts.mean_ST_All > 0 THEN 1 ELSE 0
  END AS has_st_data_team_season,

  CASE
    WHEN p.is_reg_game_week = 1
     AND ts.mean_ST_All IS NOT NULL AND ts.mean_ST_All > 0
     AND ts.sd_ST_All IS NOT NULL AND ts.sd_ST_All > 0
      THEN (p.ST_Load_All_w - ts.mean_ST_All) / ts.sd_ST_All
    ELSE 0
  END AS Z_ST_All_w,

  CASE
    WHEN p.is_reg_game_week = 1
     AND ts.mean_ST_All IS NOT NULL AND ts.mean_ST_All > 0
     AND ts.sd_ST_ScoreLinked IS NOT NULL AND ts.sd_ST_ScoreLinked > 0
      THEN (p.ST_Load_ScoreLinked_w - ts.mean_ST_ScoreLinked) / ts.sd_ST_ScoreLinked
    ELSE 0
  END AS Z_ST_ScoreLinked_w,

  CASE
    WHEN p.is_reg_game_week = 1
     AND ts.mean_ST_All IS NOT NULL AND ts.mean_ST_All > 0
     AND ts.sd_ST_NonScore IS NOT NULL AND ts.sd_ST_NonScore > 0
      THEN (p.ST_Load_NonScore_w - ts.mean_ST_NonScore) / ts.sd_ST_NonScore
    ELSE 0
  END AS Z_ST_NonScore_w

FROM panel_step5_base p
LEFT JOIN team_season_st_stats ts
  ON p.season = ts.season
 AND p.{PANEL_TEAM_COL} = ts.team
""")

We compute a rolling version of the NonScore z-score and shock flags that only looks at games already played and also keeps the model safe from "predicting" the past using future information

In [ ]:
con.execute(f"""
CREATE OR REPLACE TEMP VIEW panel_step5_rolling AS
SELECT
  *,
  AVG(ST_Load_NonScore_w) OVER (
    PARTITION BY season, {PANEL_TEAM_COL}
    ORDER BY week
    ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
  ) AS roll_mean_ST_NonScore,
  STDDEV_SAMP(ST_Load_NonScore_w) OVER (
    PARTITION BY season, {PANEL_TEAM_COL}
    ORDER BY week
    ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
  ) AS roll_sd_ST_NonScore
FROM panel_step5_base
WHERE is_reg_game_week = 1
""")

con.execute("""
CREATE OR REPLACE TEMP VIEW panel_step5_rolling_z AS
SELECT
  *,
  CASE
    WHEN roll_sd_ST_NonScore IS NOT NULL AND roll_sd_ST_NonScore > 0
      THEN (ST_Load_NonScore_w - roll_mean_ST_NonScore) / roll_sd_ST_NonScore
    ELSE 0
  END AS Z_ST_NonScore_Roll_w,
  CASE
    WHEN roll_sd_ST_NonScore IS NOT NULL AND roll_sd_ST_NonScore > 0
     AND (ST_Load_NonScore_w - roll_mean_ST_NonScore) / roll_sd_ST_NonScore >= 1
      THEN 1 ELSE 0
  END AS ST_Shock_NonScore_Roll_w
FROM panel_step5_rolling
""")

Quick sanity check to confirm that every regular season game has a valid z-score and that the values average out to approximately zero for each team's season, verifying the normalization was successful

In [ ]:
con.execute("""
SELECT
  SUM(CASE WHEN is_reg_game_week = 1 AND mean_ST_All IS NULL THEN 1 ELSE 0 END) AS reg_weeks_missing_stats,
  SUM(CASE WHEN is_reg_game_week = 1 AND Z_ST_All_w IS NULL THEN 1 ELSE 0 END) AS reg_weeks_missing_z_all,
  SUM(CASE WHEN is_reg_game_week = 1 AND Z_ST_NonScore_w IS NULL THEN 1 ELSE 0 END) AS reg_weeks_missing_z_nonscore
FROM panel_step5_z
""").df()

Quick sanity check to confirm that every regular season game has a valid z-score and that the values average out to approximately zero for each team's season, verifying the normalization was successful

In [ ]:
con.execute(f"""
SELECT
  season,
  {PANEL_TEAM_COL} AS team,
  AVG(Z_ST_All_w) AS avg_z_all,
  AVG(Z_ST_ScoreLinked_w) AS avg_z_scorelinked,
  AVG(Z_ST_NonScore_w) AS avg_z_nonscore,
  COUNT(*) AS reg_rows
FROM panel_step5_z
WHERE is_reg_game_week = 1
GROUP BY season, {PANEL_TEAM_COL}
ORDER BY ABS(AVG(Z_ST_NonScore_w)) DESC
LIMIT 25
""").df()

Quick sanity check to confirm that per week z scores have a reasonable spread and include both positive and negative values

In [ ]:
con.execute(f"""
SELECT
  season,
  week,
  {PANEL_TEAM_COL} AS team,
  ST_Load_NonScore_w,
  mean_ST_NonScore,
  sd_ST_NonScore,
  Z_ST_NonScore_w
FROM panel_step5_z
WHERE is_reg_game_week = 1
ORDER BY ABS(Z_ST_NonScore_w) DESC
LIMIT 50
""").df()

Quick sanity check to confirm that z scores are not all near zero and have plausible min and max

In [ ]:
con.execute("""
SELECT
  MIN(Z_ST_All_w) AS min_z_all,
  MAX(Z_ST_All_w) AS max_z_all,
  AVG(Z_ST_All_w) AS avg_z_all,

  MIN(Z_ST_ScoreLinked_w) AS min_z_scorelinked,
  MAX(Z_ST_ScoreLinked_w) AS max_z_scorelinked,
  AVG(Z_ST_ScoreLinked_w) AS avg_z_scorelinked,

  MIN(Z_ST_NonScore_w) AS min_z_nonscore,
  MAX(Z_ST_NonScore_w) AS max_z_nonscore,
  AVG(Z_ST_NonScore_w) AS avg_z_nonscore
FROM panel_step5_z
WHERE is_reg_game_week = 1
""").df()

We create binary shock indicators by flagging any week where the z-score exceeds 1.0, ensuring that non-regular season weeks are explicitly set to zero to avoid misclassifying byes or off-weeks as high-workload events

In [ ]:
con.execute("""
CREATE OR REPLACE TEMP VIEW panel_step5_shocks AS
SELECT
  p.*,

  CASE
    WHEN is_reg_game_week = 1 AND has_st_data_team_season = 1 AND Z_ST_All_w >= 1 THEN 1 ELSE 0
  END AS ST_Shock_All_w,

  CASE
    WHEN is_reg_game_week = 1 AND has_st_data_team_season = 1 AND Z_ST_ScoreLinked_w >= 1 THEN 1 ELSE 0
  END AS ST_Shock_ScoreLinked_w,

  CASE
    WHEN is_reg_game_week = 1 AND has_st_data_team_season = 1 AND Z_ST_NonScore_w >= 1 THEN 1 ELSE 0
  END AS ST_Shock_NonScore_w

FROM panel_step5_z p
""")

Quick sanity check to confirm that the incidence rate of the binary shock features aligns with the expected frequency of a+1σ threshold and that the is_regular_season filter is effectively preventing false positives in non-active weeks

In [ ]:
con.execute("""
SELECT
  AVG(ST_Shock_All_w * 1.0) AS shock_rate_all,
  AVG(ST_Shock_ScoreLinked_w * 1.0) AS shock_rate_scorelinked,
  AVG(ST_Shock_NonScore_w * 1.0) AS shock_rate_nonscore,
  SUM(CASE WHEN is_reg_game_week = 0 AND (ST_Shock_All_w = 1 OR ST_Shock_ScoreLinked_w = 1 OR ST_Shock_NonScore_w = 1) THEN 1 ELSE 0 END) AS shocks_on_non_reg_rows
FROM panel_step5_shocks
""").df()

We add standardized alias columns for the primary shock and z-score variables, streamlining the downstream modeling pipeline by pointing the algorithm to the NonScore metrics as the default inputs

In [ ]:
con.execute("""
CREATE OR REPLACE TEMP VIEW panel_step5_final AS
SELECT
  p.*,
  ST_Shock_NonScore_w AS ST_Shock_Primary_w,
  Z_ST_NonScore_w AS Z_ST_Primary_w
FROM panel_step5_shocks p
""")

We persist the enriched feature set to the database by overwriting the existing panel, centralizing the newly computed z-scores and shock indicators to maintain a unified data state for the rest of the workflow

In [ ]:
con.execute("""
CREATE OR REPLACE TABLE team_week_panel AS
SELECT * FROM panel_step5_final
""")

Quick sanity check to confirm that we cleaned up any duplicate columns that might have appeared during the joining process and also that each shock or z-score field only appears once in our final table

In [ ]:
helper_bases = ["has_st_data_team_season", "is_reg_game_week", "sched_game_type"]

cols = con.execute("""
SELECT column_name
FROM duckdb_columns
WHERE table_name = 'team_week_panel'
""").df()["column_name"].tolist()

dupes = [c for c in cols if any(c.startswith(base + "_") for base in helper_bases)]

print("Duplicate helper columns to drop", dupes)

if dupes:
    exclude_list = ",\n    ".join(dupes)
    con.execute(f"""
    CREATE OR REPLACE TABLE team_week_panel AS
    SELECT
      * EXCLUDE (
        {exclude_list}
      )
    FROM team_week_panel
    """)
else:
    print("No duplicate helper columns found")

Quick sanity check to confirm that every active game week shows at least some special teams activity for teams that are active and also ensuring we haven't accidentally dropped data or recorded empty games

In [ ]:
con.execute(f"""
SELECT
  season,
  {PANEL_TEAM_COL} AS team,
  week,
  game_id,
  ST_Load_All_w,
  ST_Load_ScoreLinked_w,
  ST_Load_NonScore_w
FROM team_week_panel
WHERE is_reg_game_week = 1
  AND has_st_data_team_season = 1
  AND ST_Load_All_w = 0
ORDER BY season, team, week
LIMIT 200
""").df()

Quick sanity check to confirm that row-level parity is maintained between the primary and also NonScore shock features while ensuring the pointer logic was applied accurately during the persistence phase

In [ ]:
con.execute("""
SELECT
  SUM(CASE WHEN ST_Shock_Primary_w != ST_Shock_NonScore_w THEN 1 ELSE 0 END) AS primary_mismatch_rows,
  COUNT(*) AS total_rows
FROM team_week_panel
""").df()

Quick sanity check to confirm that the physical schema of the team-week panel has been updated to include the full suite of normalization metrics and also binary shock indicators

In [ ]:
final_cols = set(con.execute("DESCRIBE team_week_panel").df()["column_name"].tolist())

need_now = [
    "mean_ST_All",
    "sd_ST_All",
    "mean_ST_ScoreLinked",
    "sd_ST_ScoreLinked",
    "mean_ST_NonScore",
    "sd_ST_NonScore",
    "Z_ST_All_w",
    "Z_ST_ScoreLinked_w",
    "Z_ST_NonScore_w",
    "ST_Shock_All_w",
    "ST_Shock_ScoreLinked_w",
    "ST_Shock_NonScore_w",
    "ST_Shock_Primary_w",
    "Z_ST_Primary_w",
]

missing_now = [c for c in need_now if c not in final_cols]

print("Missing columns", missing_now)
print("OK" if not missing_now else "STOP, Step 5 columns missing from stored panel")

Quick sanity check to confirm that the table schema matches our intended blueprint exactly and also that any "garbage" columns from the join process were dropped before the final write

In [ ]:
con.execute("""
SELECT column_name
FROM duckdb_columns
WHERE table_name = 'team_week_panel'
  AND (column_name LIKE '%sched%' OR column_name LIKE 'is_reg%' OR column_name LIKE 'has_st%')
ORDER BY column_name
""").df()